In [1]:
from collections import OrderedDict    # For recording the model specification 

import pandas as pd                    # For file input/output
import numpy as np                     # For vectorized math operations

import pylogit as pl                   # For choice model estimation
import pickle

In [2]:
catsup = pd.read_csv('data/catsup_trainformat.csv')
catsup.head()

,chosen,id,alt,display,feature,price,chid
0,0,1,0,0,0,4.6,0
1,0,1,1,0,0,3.7,0
2,1,1,2,0,0,5.2,0
3,0,1,3,0,0,3.4,0
4,0,1,0,0,0,4.6,1


In [10]:
infile = open('./data/10_dgp.p', 'rb')
big_dict = pickle.load(infile)
Y_dgp = big_dict['theta: [ 1.5  1.  -1.1  0.8  0.1  1.2]']
print(Y_dgp.shape)
chosen = np.zeros([11192,1])
j = 0
for choice in Y_dgp[:,0]:
    for i in range(4):
        if i == choice:
            chosen[i+j] = True
        else:
            chosen[i+j] = False
    j+=4
catsup['chosen'] = chosen
catsup.head(30)

(2798, 10)


,chosen,id,alt,display,feature,price,chid
0,0.0,1,0,0,0,4.6,0
1,0.0,1,1,0,0,3.7,0
2,0.0,1,2,0,0,5.2,0
3,1.0,1,3,0,0,3.4,0
4,0.0,1,0,0,0,4.6,1
5,1.0,1,1,0,0,4.3,1
6,0.0,1,2,0,0,5.2,1
7,0.0,1,3,0,0,4.4,1
8,0.0,1,0,0,0,4.6,2
9,1.0,1,1,0,1,2.5,2


In [11]:
#get variable list
index_var_names = ['display','feature','price']
for col in index_var_names:
    catsup[col] = catsup[col].astype(float)

In [12]:
#specification
example_specification = OrderedDict()
example_names = OrderedDict()

# Note that the names used below are simply for consistency with
# the coefficient names given in the mlogit vignette.
for col in index_var_names:
    example_specification[col] = [[0, 1, 2, 3]]
    example_names[col] = [col]

In [13]:
# Provide the module with the needed input arguments to create
# an instance of the Mixed Logit model class.

# Note that "chid" is used as the obs_id_col because "chid" is
# the choice situation id.

# Currently, the obs_id_col argument name is unfortunate because
# in the most general of senses, it refers to the situation id.
# In panel data settings, the mixing_id_col argument is what one 
# would generally think of as a "observation id".

# For mixed logit models, the "mixing_id_col" argument specifies
# the units of observation that the coefficients are randomly
# distributed over.
example_mixed = pl.create_choice_model(data=catsup,
                                       alt_id_col="alt",
                                       obs_id_col="chid",
                                       choice_col="chosen",
                                       specification=example_specification,
                                       model_type="Mixed Logit",
                                       names=example_names,
                                       mixing_id_col="id",
                                       mixing_vars=index_var_names)

# Note 2 * len(index_var_names) is used because we are estimating
# both the mean and standard deviation of each of the random coefficients
# for the listed index variables.
example_mixed.fit_mle(init_vals=np.zeros(2 * len(index_var_names)),
                      num_draws=600,
                      seed=123)

# Look at the estimated results
example_mixed.get_statsmodels_summary()

Log-likelihood at zero: -3,878.8516
Initial Log-likelihood: -3,878.8516


C:\Users\niels\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:507: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time for Point Estimation: 1.68 minutes.
Final log-likelihood: -563.3480


C:\Users\niels\Anaconda3\lib\site-packages\pylogit\base_multinomial_cm_v2.py:1259: RuntimeWarning: invalid value encountered in sqrt
  self._store_inferential_results(np.sqrt(np.diag(self.robust_cov)),
C:\Users\niels\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\niels\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\niels\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Mixed Logit Model Regression Results                     
==============================================================================
Dep. Variable:                 chosen   No. Observations:                2,798
Model:              Mixed Logit Model   Df Residuals:                    2,792
Method:                           MLE   Df Model:                            6
Date:                Tue, 18 Jun 2019   Pseudo R-squ.:                   0.855
Time:                        20:29:04   Pseudo R-bar-squ.:               0.853
AIC:                        1,138.696   Log-Likelihood:               -563.348
BIC:                        1,174.316   LL-Null:                    -3,878.852
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
display          84.5674     23.373      3.618      0.000      38.757     130.378
feature          74.9100     20.966      3.573      0.000      33.818     116.002
price             9.5527      8.402      1.137      0.256      -6.915      26.020
Sigma display    -0.8265      9.184     -0.090      0.928     -18.827      17.174
Sigma feature    14.3573     10.843      1.324      0.185      -6.894      35.608
Sigma price      76.7933     23.137      3.319      0.001      31.446     122.141
=================================================================================
"""

In [ ]:
example_mixed.